<a href="https://colab.research.google.com/github/patrycjalazna/transformers/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importy💅🏻💅🏻💅🏻

In [ ]:
!pip install 'transformers==4.12.5' 'tokenizers==0.10.3' 'sentencepiece==0.1.96' 'datasets==1.16.1' 'accelerate==0.5.1' 'sacremoses==0.0.46' 'sacrebleu==2.0.0';

In [ ]:
import torch
from torch import nn
from torch.nn import MSELoss, CrossEntropyLoss, BCEWithLogitsLoss
from transformers import RobertaForSequenceClassification, RobertaModel
from transformers.modeling_outputs import SequenceClassifierOutput
import json
from pathlib import Path
from typing import Dict, List
from datasets import load_dataset

## 🤗 Dataset

Dataset *emotion* jest zbiorem danych angielskich wiadomości na Twitterze zawierających sześć podstawowych emocji: gniew, strach, radość, miłość, smutek i zaskoczenie.

Link do datasetu: [hugginface](https://huggingface.co/datasets/emotion)

Przykład:

```
{
    "label": 0,
    "text": "im feeling quite sad and sorry for myself but ill snap out of it soon"
}
```



In [ ]:
dataset = load_dataset('emotion')

Using custom data configuration default
Reusing dataset emotion (/root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)


  0%|          | 0/3 [00:00<?, ?it/s]

Dane mamy automatycznie podzielone na train/val/test w stosunku 8:1:1.

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


Następnie tworzymy folder, w którym zapiszemy dane.

In [ ]:
!mkdir -v -p data

mkdir: created directory 'data'


In [ ]:
train_path = Path('data/train.json')
valid_path = Path('data/valid.json')
test_path = Path('data/test.json')

In [ ]:
data_train, data_valid, data_test = [], [], []

In [ ]:
for source_data, dataset, max_size in [
  (dataset['train'], data_train, None),
  (dataset['test'], data_valid, None)
]:
  for i, data in enumerate(source_data):
    if max_size is not None and i >= max_size:
      break
    data_line = {
      'label': int(data['label']),
      'text': data['text'],
    }
    dataset.append(data_line)

print(f'Train: {len(data_train):6d}')

Train:  16000


In [ ]:
data_class_1, data_class_2 = [], []

for data in data_valid:
  label = data['label']
  if label == 0:
    data_class_1.append(data)
  elif label == 1:
    data_class_2.append(data)

print(f'Label 1: {len(data_class_1):6d}')
print(f'Label 2: {len(data_class_2):6d}')

size_half_class_1 = int(len(data_class_1) / 2)
size_half_class_2 = int(len(data_class_2) / 2)

data_valid = data_class_1[:size_half_class_1] + data_class_2[:size_half_class_2]
data_test = data_class_1[size_half_class_1:] + data_class_2[size_half_class_2:]

print(f'Valid: {len(data_valid):6d}')
print(f'Test : {len(data_test):6d}')

Label 1:    581
Label 2:    695
Valid:    637
Test :    639


In [ ]:
MAP_LABEL_TRANSLATION = {
    0: 'negative',
    1: 'positive',
}

In [ ]:
def save_as_translations(original_save_path: Path, data_to_save: List[Dict]) -> None:
    file_name = 'translations-' + original_save_path.name
    file_path = original_save_path.parent / file_name

    print(f'Saving into: {file_path}')
    with open(file_path, 'wt') as f_write:
        for data_line in data_to_save:
            label = data_line['label']
            new_label = MAP_LABEL_TRANSLATION[label]
            data_line['label'] = new_label
            data_line_str = json.dumps(data_line)
            f_write.write(f'{data_line_str}\n')

In [ ]:
for file_path, data_to_save in [(train_path, data_train), (valid_path, data_valid), (test_path, data_test)]:
  print(f'Saving into: {file_path}')
  with open(file_path, 'wt') as f_write:
    for data_line in data_to_save:
      data_line_str = json.dumps(data_line)
      f_write.write(f'{data_line_str}\n')

  save_as_translations(file_path, data_to_save)

Saving into: data/train.json
Saving into: data/translations-train.json


KeyError: ignored

poprawic, bo cos nie bangla

In [ ]:
data_class_1, data_class_2 = [], []

for data in data_valid:
  label = data['label']
  if label == 0:
    data_class_1.append(data)
  elif label == 1:
    data_class_2.append(data)

print(f'Label 1: {len(data_class_1):6d}')
print(f'Label 2: {len(data_class_2):6d}')

size_half_class_1 = int(len(data_class_1) / 2)
size_half_class_2 = int(len(data_class_2) / 2)

data_valid = data_class_1[:size_half_class_1] + data_class_2[:size_half_class_2]
data_test = data_class_1[size_half_class_1:] + data_class_2[size_half_class_2:]

print(f'Valid: {len(data_valid):6d}')
print(f'Test : {len(data_test):6d}')

Label 1:    290
Label 2:    347
Valid:    318
Test :    319
